# Lab 2 - Exploring Text Data

### Eric Smith and Jake Carlson

## Introduction
For this lab we will be examining questions and answers on the popular programming Q&A website, Stack Overflow. Stack Overflow allows programmers who are stuck on a problem ask the community about ways to resolve or circumvent said problem. This helps people write more accurate code faster. However, the community can be pretty hard on those who do not fully understand their question or are disrespectful. It could also be the case that the few developers who have experience with a problem similar to yours are not on the website when you post your question. If too much time passes, your post may be burried forever. This makes for some interesing questions. When should you make your post so that it has the highest chance of being answered? Are there specific keywords or phasing that encourage other programmers to answer your questions? Are there specific languages that have their questions answered faster than others? These are questions we will be exploring int his lab.

## Business Understanding

### Motivations
Stack Overflow is a go-to resource for developers. Questions are often distilled down to a single block of code that is easily digestible, and can be matched to the question you came to the site for. However, if you're truly stuck on something, it can seem like an eternity of waiting before someone responds to your question, and sometimes, it may not be answered at all. If you're working in industry and spending a lot of time waiting for a question to be answered, you could fall behind schedule and miss deadlines. These delays could cost your company a great deal in lost revenue and tech debt. Therefore, accurately predicting the amount of time to get a question answered can be seen as a valuable project management tool.

### Objectives
It would be useful if you could predict how quickly your question will be answered if you post it at various times through out the day. For a prediction tool like this to be useful, we want something that is 80-95% accurate in predicting when a question will be answered by. As a project manager, you want your developers to be asking questions that are concise and respectful. Therefore, a tool that could recommend keywords to add to your post to make it more attractive, and provide you with approximate gains in time for reformatting, would also be a valuable project management tool.

## Data Understanding

### Data Attributes
The following is a list of attributes in the data, their data types, and a brief description of the attribute.

#### Questions
- **Id** (nominal): A unique identifier for each question
- **OwnerUserId** (nominal): A unique identifier for the person who posted the question
- **CreationDate** (ordinal): A timestamp of when the question was posted
- **ClosedDare** (ordinal): A timestamp of when the question was closed, if the question wasn't closed this field is NaN
- **Score** (ordinal): ratio?? The number of upvotes a post has
- **Title** (text): A title for the question
- **Body** (text): The question body

#### Answers
- **Id** (nominal): A unique identifier for each answer
- **OwnerUserId** (nominal): A unique identifier for the person who posted the answer
- **CreationDate** (ordinal): A timestamp of when the answer was posted
- **ParentId** (nominal): The unique identifier of the question this answer is for
- **Score** (ordinal): ratio?? The number of upvotes an answer has
- **Body** (text): The answer body


## Data Quality
Our data it too big, so we will take a rendom subsample of 40% of the original data set.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# read data
df = pd.read_csv('./data/Questions.csv', encoding='ISO-8859-1')

In [3]:
df.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264216 entries, 0 to 1264215
Data columns (total 7 columns):
Id              1264216 non-null int64
OwnerUserId     1249762 non-null float64
CreationDate    1264216 non-null object
ClosedDate      55959 non-null object
Score           1264216 non-null int64
Title           1264216 non-null object
Body            1264216 non-null object
dtypes: float64(1), int64(2), object(4)
memory usage: 67.5+ MB


Trial and error has shown us that 1,264,000 elements is too many to work with. Therefore, we will use a random sample to reduce our data set size to 12,000 elements.

In [5]:
df = df.sample(n=12000, replace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 406590 to 598237
Data columns (total 7 columns):
Id              12000 non-null int64
OwnerUserId     11856 non-null float64
CreationDate    12000 non-null object
ClosedDate      563 non-null object
Score           12000 non-null int64
Title           12000 non-null object
Body            12000 non-null object
dtypes: float64(1), int64(2), object(4)
memory usage: 750.0+ KB


The body of each question is formatted in HTML, so we will need to parse through each body and remove the HTML tags. We also need to replace HTML entities with their character representations. Finally, many questions have accompanying code blocks, so we will parse out the code blocks into a new attribute for each entry.

In [ ]:
from bs4 import BeautifulSoup
import html
import re

# compile regexes
tag_re = re.compile('<[^<]+?>')
newline_re = re.compile('\n+')

def clean_html(body_text):
    soup = BeautifulSoup(body_text, 'html.parser')
    code_tags = soup.findAll('code')
    code_text = ""
    
    # unescape html entities
    body_text = html.unescape(body_text)
    
    # remove code blocks, saving blocks to new string
    for c in code_tags:
        body_text = body_text.replace("<code>{}</code>".format(c.string),
                                      '')
        if c.string:
            code_text += c.string

    # remove remaining tags and multiple newlines
    body_text = tag_re.sub('', body_text)
    body_text = newline_re.sub('\n', body_text)
    
    return body_text, code_text

as_list = df.Body.tolist()
body_list = []
code_list = []
for i in range(len(as_list)):
    body_text, code_text = clean_html(as_list[i])
    body_list.append(body_text)
    code_list.append(code_text)

# update body column
df = df.assign(Body=body_list)
# add code column
df = df.assign(Code=code_list)
df.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Code
406590,14372690,1820022.0,2013-01-17T05:25:30Z,NaN,2,temporarily disable all events in javascript,How do you temporarily disable all events in j...,
665037,22761320,3234665.0,2014-03-31T12:15:56Z,NaN,0,URL Rewriting is showing 404 Error after Upgra...,I have just change framework of my website fro...,http://www.theprojectjugaad.com/PlacementAssis...
640709,22000080,3231979.0,2014-02-24T22:05:14Z,NaN,1,Get the source url of link by custom firefox M...,I have tried to: \nAdd menu item to the mozill...,"var contextMenu = require(""sdk/context-menu"")..."
151372,5971910,734248.0,2011-05-11T23:59:01Z,NaN,0,"Obtaining right value from ""JSON-style"" NSStri...","I am trying to get the NSString ""Bonjour"" from...","[[[""Bonjour"",""Hello"","""",""""]],,""en"",,[[""Bonjour..."
293479,10695110,1316405.0,2012-05-22T02:43:12Z,NaN,0,specify index variables if equal to self,"I'm sure the wording was completely wrong, sor...","function playArray(ele, array) {\n ..."


With the HTML tags removed and the code isolated, the question body is much cleaner. Now lets add another column for the time in minutes it took to get the question answered. For this value, we will look for the highest rated answer for each question and use the time when that answer was posted.

In [ ]:
df_ans = pd.read_csv('./data/Answers.csv', encoding='ISO-8859-1')
df_ans.head()

In [ ]:
df_ans.info()

In [ ]:
ids = df.Id.unique()
df_ans = df_ans[df_ans.ParentId.isin(ids)]
df_ans.info()

We will clean the answers data frame using the same function as the questions data frame.

In [ ]:
as_list = df_ans.Body.tolist()
body_list = []
code_list = []
for i in range(len(as_list)):
    body_text, code_text = clean_html(as_list[i])
    body_list.append(body_text)
    code_list.append(code_text)

# update body column
df_ans = df_ans.assign(Body=body_list)
# add code column
df_ans = df_ans.assign(Code=code_list)
df_ans.head()

In [ ]:
# adapted from https://stackoverflow.com/questions/2788871/date-difference-in-minutes-in-python
from datetime import datetime
import time

fmt = '%Y-%m-%dT%H:%M:%SZ'
def get_minutes_diff(timestamp1, timestamp2):
#     if np.isnan(x.ClosedDate):
#         return np.nan
    d1 = datetime.strptime(timestamp1, fmt)
    d2 = datetime.strptime(timestamp2, fmt)
    
    # convert to unix timestamp
    d1_ts = time.mktime(d1.timetuple())
    d2_ts = time.mktime(d2.timetuple())
    
    # convert to minutes
    return int(d2_ts - d1_ts) / 60

durations = []
for i, row in df.iterrows():
    answer = df_ans[ df_ans.ParentId == row.Id ]
    answer = answer[ answer.Score == answer.Score.max() ]
    answer_time = answer.CreationDate.values
    if len(answer_time) > 0:
        answer_time = answer_time[0]
        df.loc[i, 'Duration'] = get_minutes_diff(row.CreationDate, 
                                                  answer_time)

df.head()

Load the tags and save the tags that correspond to a question.

In [ ]:
df_tags = pd.read_csv('./data/Tags.csv', encoding='ISO-8859-1')
df_tags.head()

In [ ]:
ids = df.Id.unique()
df_tags = df_tags[df_tags.Id.isin(ids)]
df_tags.head()

Save the cleaned data to a persistance file.

In [ ]:
df.to_csv('./clean-data/Questions.csv', sep=',')
df_ans.to_csv('./clean-data/Answers.csv', sep=',')
df_tags.to_csv('./clean-data/Tags.csv', sep=',')

## Data Visualization

In [ ]:
df = pd.read_csv('./clean-data/Questions.csv')
df_ans = pd.read_csv('./clean-data/Answers.csv')
df_tags = pd.read_csv('./clean-data/Tags.csv')

In [ ]:
df_tags.info()

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

tag_counts = Counter(df_tags.Tag)

# remove tags that have less than 175 occurances
for t in list(tag_counts):
    if tag_counts[t] < 175:
        del tag_counts[t]

df_tag_counts = pd.DataFrame.from_dict(tag_counts, orient='index')
df_tag_counts.plot(kind='barh', fontsize=12, figsize=(8,6))
plt.xlabel('Number of occurances', fontsize=14)
plt.title('Tags', fontsize=18)
plt.legend([])
plt.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words='english')
bag_words = count_vect.fit_transform(df.Title.tolist())

In [ ]:
len(count_vect.vocabulary_)

In [ ]:
df_title = pd.DataFrame(data=bag_words.toarray(), 
                        columns=count_vect.get_feature_names())

In [ ]:
# print 10 most common words
df_title.sum().sort_values()[-10:]

In [ ]:
# adapted from https://stackoverflow.com/a/16359854
# 1 day = 86400 seconds
# 1 week = 604800 seconds
df_durations = pd.cut(df.Duration, bins=[-1, 86400, 604800], labels=False)
labels = np.array('< 1 day', '< 1 week', '> 1 week')

df_durations